Toxicity Dataset : https://archive.ics.uci.edu/dataset/728/toxicity-2

The dataset includes 171 molecules designed for functional domains of a core clock protein, CRY1, responsible for generating circadian rhythm. 56 of the molecules are toxic and the rest are non-toxic. 

The data consists a complete set of 1203 molecular descriptors and needs feature selection before classification since some of the features are redundant. 

Introductory Paper:
Structure-based design and classifications of small molecules regulating the circadian rhythm period
By Seref Gul, F. Rahim, Safak Isin, Fatma Yilmaz, Nuri Ozturk, M. Turkay, I. Kavakli. 2021
https://www.semanticscholar.org/paper/Structure-based-design-and-classifications-of-small-Gul-Rahim/5944836c47bc7d1a2b0464a9a1db94d4bc7f28ce
Published in Scientific reports

# Import necessary libraries

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')


# Load the toxicity dataset

In [17]:
import pandas as pd

# Read the CSV file
data = pd.read_csv("./data.csv")

# Display basic information about the dataset
print("Dataset shape:", data.shape)
print("\nFirst few rows:")
print(data.head())
print("\nColumn names:")
print(data.columns.tolist())

# Separate features and target
# Assuming the last column or a column named 'Class' contains the target
if 'Class' in data.columns:
    X = data.drop('Class', axis=1)
    y = data['Class']
else:
    # Assume last column is the target
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

Dataset shape: (171, 1204)

First few rows:
   MATS3v  nHBint10  MATS3s  MATS3p  nHBDon_Lipinski  minHBint8  MATS3e  \
0  0.0908         0  0.0075  0.0173                0        0.0 -0.0436   
1  0.0213         0  0.1144 -0.0410                0        0.0  0.1231   
2  0.0018         0 -0.0156 -0.0765                2        0.0 -0.1138   
3 -0.0251         0 -0.0064 -0.0894                3        0.0 -0.0747   
4  0.0135         0  0.0424 -0.0353                0        0.0 -0.0638   

   MATS3c  minHBint2  MATS3m  ...   WTPT-4   WTPT-5  ETA_EtaP_L  ETA_EtaP_F  \
0  0.0409        0.0  0.1368  ...   0.0000   0.0000      0.1780      1.5488   
1 -0.0316        0.0  0.1318  ...   8.8660  19.3525      0.1739      1.3718   
2 -0.1791        0.0  0.0615  ...   5.2267  27.8796      0.1688      1.4395   
3 -0.1151        0.0  0.0361  ...   7.7896  24.7336      0.1702      1.4654   
4  0.0307        0.0  0.0306  ...  12.3240  19.7486      0.1789      1.4495   

   ETA_EtaP_B  nT5Ring  SHdNH 

# EDA

In [18]:
# Basic data exploration
print("\n=== DATA EXPLORATION ===")
print(f"Shape of features (X): {X.shape}")
print(f"Shape of target (y): {y.shape}")
print(f"\nTarget distribution:")
print(y.value_counts())
print(f"\nClass balance:")
print(y.value_counts(normalize=True))

# Check target data type and unique values
print(f"\nTarget data type: {y.dtype}")
print(f"Unique target values: {y.unique()}")


=== DATA EXPLORATION ===
Shape of features (X): (171, 1203)
Shape of target (y): (171,)

Target distribution:
Class
NonToxic    115
Toxic        56
Name: count, dtype: int64

Class balance:
Class
NonToxic    0.672515
Toxic       0.327485
Name: proportion, dtype: float64

Target data type: object
Unique target values: ['NonToxic' 'Toxic']


In [19]:
# Check for missing values
print(f"\nMissing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in target: {y.isnull().sum()}")



Missing values in features: 0
Missing values in target: 0


# Preprocessing

In [20]:
# Handle missing values if any
print("=== PREPROCESSING ===")
print(f"Missing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in target: {y.isnull().sum()}")

if X.isnull().sum().sum() > 0:
    # Option 1: Drop columns with too many missing values
    missing_threshold = 0.3  # Drop columns with >30% missing
    missing_prop = X.isnull().sum() / len(X)
    cols_to_drop = missing_prop[missing_prop > missing_threshold].index
    X = X.drop(columns=cols_to_drop)
    
    # Option 2: Impute remaining missing values
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    print(f"Missing values imputed")

# Convert target to binary (1 for NonToxic, 0 for Toxic) - FLIPPED LABELS
y_binary = (y == 'NonToxic').astype(int)

# Verify the binary conversion
print("\nBinary target distribution:")
print(y_binary.value_counts())
print(f"Class balance: {y_binary.value_counts(normalize=True)}")

# Double-check the conversion is correct
print(f"\nMapping verification:")
print(f"Original 'NonToxic' → Binary 1: {y_binary[y == 'NonToxic'].unique()}")
print(f"Original 'Toxic' → Binary 0: {y_binary[y == 'Toxic'].unique()}")

=== PREPROCESSING ===
Missing values in features: 0
Missing values in target: 0

Binary target distribution:
Class
1    115
0     56
Name: count, dtype: int64
Class balance: Class
1    0.672515
0    0.327485
Name: proportion, dtype: float64

Mapping verification:
Original 'NonToxic' → Binary 1: [1]
Original 'Toxic' → Binary 0: [0]


In [21]:
# Feature preprocessing
print("\n=== FEATURE PREPROCESSING ===")

# Remove constant features
constant_filter = VarianceThreshold(threshold=0)
X_filtered = constant_filter.fit_transform(X)
constant_columns = X.columns[~constant_filter.get_support()]
print(f"Removed {len(constant_columns)} constant features")

# Remove quasi-constant features (variance < 0.01)
quasi_constant_filter = VarianceThreshold(threshold=0.01)
X_filtered = quasi_constant_filter.fit_transform(X_filtered)
selected_features = X.columns[constant_filter.get_support()][quasi_constant_filter.get_support()]
X_filtered = pd.DataFrame(X_filtered, columns=selected_features)
print(f"Remaining features after variance filtering: {X_filtered.shape[1]}")

# Remove highly correlated features
correlation_matrix = X_filtered.corr().abs()
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)
high_corr_features = [column for column in upper_triangle.columns 
                      if any(upper_triangle[column] > 0.95)]
X_filtered = X_filtered.drop(columns=high_corr_features)
print(f"Removed {len(high_corr_features)} highly correlated features")
print(f"Final feature count: {X_filtered.shape[1]}")


=== FEATURE PREPROCESSING ===
Removed 0 constant features
Remaining features after variance filtering: 994
Removed 434 highly correlated features
Final feature count: 560


# Data Splitting

In [22]:
# Split the data with stratification to ensure balanced folds
from sklearn.model_selection import StratifiedKFold

# Add some randomness to address potential ordering issues
np.random.seed(42)
shuffle_idx = np.random.permutation(len(X_filtered))
X_shuffled = X_filtered.iloc[shuffle_idx].reset_index(drop=True)
y_shuffled = y_binary.iloc[shuffle_idx].reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    X_shuffled, y_shuffled, test_size=0.2, random_state=42, 
    stratify=y_shuffled, shuffle=True
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train_scaled.shape}")
print(f"Test set size: {X_test_scaled.shape}")

# Check class distribution in train and test sets
print(f"\nTrain set class distribution:")
print(pd.Series(y_train).value_counts(normalize=True))
print(f"\nTest set class distribution:")
print(pd.Series(y_test).value_counts(normalize=True))

Training set size: (136, 560)
Test set size: (35, 560)

Train set class distribution:
Class
1    0.669118
0    0.330882
Name: proportion, dtype: float64

Test set class distribution:
Class
1    0.685714
0    0.314286
Name: proportion, dtype: float64


# Evaluation function

In [23]:
# Define evaluation function
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Evaluate a classification model and return metrics"""
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probabilities for AUC
    if hasattr(model, 'predict_proba'):
        y_train_proba = model.predict_proba(X_train)[:, 1]
        y_test_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_train_proba = model.decision_function(X_train)
        y_test_proba = model.decision_function(X_test)
    
    # Calculate metrics
    metrics = {
        'model': model_name,
        'train_accuracy': accuracy_score(y_train, y_train_pred),
        'test_accuracy': accuracy_score(y_test, y_test_pred),
        'train_auc': roc_auc_score(y_train, y_train_proba),
        'test_auc': roc_auc_score(y_test, y_test_proba),
        'precision': precision_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred),
        'f1': f1_score(y_test, y_test_pred)
    }
    
    return metrics, y_test_pred, y_test_proba

In [24]:
# Initialize results storage
results = []
all_predictions = {}
all_probabilities = {}

In [25]:
print("\n" + "="*80)
print("COMPREHENSIVE MODEL COMPARISON: ORDINARY VS PENALIZED REGRESSION")
print("="*80)

print("""
This analysis compares the following models:
1. Ordinary Logistic Regression (no regularization) - Baseline
2. Ridge Regression (L2 penalty) - Shrinks coefficients
3. Lasso Regression (L1 penalty) - Feature selection + shrinkage  
4. Elastic Net (L1 + L2 penalty) - Combines both approaches
""")


COMPREHENSIVE MODEL COMPARISON: ORDINARY VS PENALIZED REGRESSION

This analysis compares the following models:
1. Ordinary Logistic Regression (no regularization) - Baseline
2. Ridge Regression (L2 penalty) - Shrinks coefficients
3. Lasso Regression (L1 penalty) - Feature selection + shrinkage  
4. Elastic Net (L1 + L2 penalty) - Combines both approaches



# Model training and evaluation

## Ordinary Logistic Regression

In [28]:
# 0. Ordinary Logistic Regression (Baseline)
print("\n0. ORDINARY LOGISTIC REGRESSION (BASELINE)")
print("-" * 50)

# No regularization - this is our baseline to compare against penalized methods
ordinary_lr = LogisticRegression(
    penalty=None, 
    max_iter=5000, 
    solver='lbfgs'
    )
ordinary_lr.fit(X_train_scaled, y_train)

# Evaluate ordinary logistic regression
ordinary_metrics, ordinary_pred, ordinary_proba = evaluate_model(
    ordinary_lr, X_train_scaled, X_test_scaled, y_train, y_test, 'Ordinary LR'
)
results.append(ordinary_metrics)
all_predictions['Ordinary LR'] = ordinary_pred
all_probabilities['Ordinary LR'] = ordinary_proba

print(f"Training Accuracy: {ordinary_metrics['train_accuracy']:.4f}")
print(f"Test Accuracy: {ordinary_metrics['test_accuracy']:.4f}")
print(f"Test AUC: {ordinary_metrics['test_auc']:.4f}")
print(f"Precision: {ordinary_metrics['precision']:.4f}")
print(f"Recall: {ordinary_metrics['recall']:.4f}")
print(f"F1-Score: {ordinary_metrics['f1']:.4f}")

# Check for overfitting
overfitting = ordinary_metrics['train_accuracy'] - ordinary_metrics['test_accuracy']
print(f"Overfitting Gap (Train - Test Accuracy): {overfitting:.4f}")
if overfitting > 0.05:
    print("⚠️  Significant overfitting detected - penalized methods should help!")
else:
    print("✓ Low overfitting - but regularization may still improve generalization")


0. ORDINARY LOGISTIC REGRESSION (BASELINE)
--------------------------------------------------
Training Accuracy: 1.0000
Test Accuracy: 0.6286
Test AUC: 0.5909
Precision: 0.7619
Recall: 0.6667
F1-Score: 0.7111
Overfitting Gap (Train - Test Accuracy): 0.3714
⚠️  Significant overfitting detected - penalized methods should help!


## Ridge Regression

In [ ]:
# 1. Ridge Regression (L2 Regularization)
print("\n1. RIDGE REGRESSION (L2 REGULARIZATION)")
print("-" * 50)

ridge = LogisticRegressionCV(
    Cs=50, # number of C values or specify a list
    cv=5, # number of cross-validation folds
    penalty='l2',
    solver='lbfgs', # or 'liblinear', 'saga'
    scoring='accuracy', # or other metrics
    max_iter=1000,
    n_jobs=-1 # use all available cores
)
ridge.fit(X_train_scaled, y_train)

# Evaluate ridge regression
ridge_metrics, ridge_pred, ridge_proba = evaluate_model(
    ridge, X_train_scaled, X_test_scaled, y_train, y_test, 'Ridge LR'
)
results.append(ridge_metrics)
all_predictions['Ridge LR'] = ridge_pred
all_probabilities['Ridge LR'] = ridge_proba
print(f"Training Accuracy: {ridge_metrics['train_accuracy']:.4f}")
print(f"Test Accuracy: {ridge_metrics['test_accuracy']:.4f}")
print(f"Test AUC: {ridge_metrics['test_auc']:.4f}")
print(f"Precision: {ridge_metrics['precision']:.4f}")
print(f"Recall: {ridge_metrics['recall']:.4f}")
print(f"F1-Score: {ridge_metrics['f1']:.4f}")



1. RIDGE REGRESSION (L2 REGULARIZATION)
--------------------------------------------------
Training Accuracy: 0.6691
Test Accuracy: 0.6857
Test AUC: 0.5492
Precision: 0.6857
Recall: 1.0000
F1-Score: 0.8136


## Lasso Regression

In [ ]:
# 2. Lasso Regression (L1 Regularization)
print("\n2. LASSO REGRESSION (L1 REGULARIZATION)")
print("-" * 60)

lasso = LogisticRegressionCV(
    Cs=50, # number of C values or a list of C values
    cv=5, # number of cross-validation folds
    penalty='l1',
    solver='saga',
    scoring='accuracy', # or other metrics
    max_iter=1000, # recommended higher value for convergence
    n_jobs=-1 # use all cores
)
lasso.fit(X_train_scaled, y_train)

# Evaluate lasso regression
lasso_metrics, lasso_pred, lasso_proba = evaluate_model(
    lasso, X_train_scaled, X_test_scaled, y_train, y_test, 'Lasso LR'
)
results.append(lasso_metrics)
all_predictions['Lasso LR'] = lasso_pred
all_probabilities['Lasso LR'] = lasso_proba
print(f"Training Accuracy: {lasso_metrics['train_accuracy']:.4f}")
print(f"Test Accuracy: {lasso_metrics['test_accuracy']:.4f}")
print(f"Test AUC: {lasso_metrics['test_auc']:.4f}")
print(f"Precision: {lasso_metrics['precision']:.4f}")
print(f"Recall: {lasso_metrics['recall']:.4f}")
print(f"F1-Score: {lasso_metrics['f1']:.4f}")



2. LASSO REGRESSION (L1 REGULARIZATION) - IMPROVED ANALYSIS
------------------------------------------------------------
Training Accuracy: 0.6691
Test Accuracy: 0.6857
Test AUC: 0.5000
Precision: 0.6857
Recall: 1.0000
F1-Score: 0.8136


## Elastic Net

In [34]:
# 3. Elastic Net (L1 + L2 Regularization)
print("\n3. ELASTIC NET (L1 + L2 REGULARIZATION)")
print("-" * 50)

elastic_net = LogisticRegressionCV(
    Cs=10, # number of C values or a list of C values
    cv=5, # number of cross-validation folds
    penalty='elasticnet',
    solver='saga',
    l1_ratios=[0.1, 0.5, 0.7, 0.9], # mix of L1 and L2
    scoring='accuracy', # or other metrics
    max_iter=1000,
    n_jobs=-1 # use all cores
)
elastic_net.fit(X_train_scaled, y_train)

# Evaluate elastic net regression
elastic_net_metrics, elastic_net_pred, elastic_net_proba = evaluate_model(
    elastic_net, X_train_scaled, X_test_scaled, y_train, y_test, 'Elastic Net LR'
)
results.append(elastic_net_metrics)
all_predictions['Elastic Net LR'] = elastic_net_pred
all_probabilities['Elastic Net LR'] = elastic_net_proba
print(f"Training Accuracy: {elastic_net_metrics['train_accuracy']:.4f}")
print(f"Test Accuracy: {elastic_net_metrics['test_accuracy']:.4f}")
print(f"Test AUC: {elastic_net_metrics['test_auc']:.4f}")
print(f"Precision: {elastic_net_metrics['precision']:.4f}")
print(f"Recall: {elastic_net_metrics['recall']:.4f}")
print(f"F1-Score: {elastic_net_metrics['f1']:.4f}")



3. ELASTIC NET (L1 + L2 REGULARIZATION)
--------------------------------------------------
Training Accuracy: 0.6691
Test Accuracy: 0.6857
Test AUC: 0.5000
Precision: 0.6857
Recall: 1.0000
F1-Score: 0.8136


# Comparison of models

In [24]:
# Create comprehensive results comparison
results_df = pd.DataFrame(results)

print("\n" + "="*80)
print("COMPREHENSIVE MODEL COMPARISON RESULTS")
print("="*80)

# Display results table
print("\nPerformance Metrics Table:")
print(results_df.round(4))

# Calculate relative improvements over ordinary logistic regression
print("\nRelative Improvements over Ordinary Logistic Regression:")
baseline_metrics = results_df[results_df['model'] == 'Ordinary LR'].iloc[0]
for idx, row in results_df.iterrows():
    if row['model'] != 'Ordinary LR':
        auc_improvement = row['test_auc'] - baseline_metrics['test_auc']
        acc_improvement = row['test_accuracy'] - baseline_metrics['test_accuracy']
        overfitting_reduction = (baseline_metrics['train_accuracy'] - baseline_metrics['test_accuracy']) - \
                               (row['train_accuracy'] - row['test_accuracy'])
        print(f"\n{row['model']}:")
        print(f"  AUC improvement: {auc_improvement:+.4f}")
        print(f"  Accuracy improvement: {acc_improvement:+.4f}")
        print(f"  Overfitting reduction: {overfitting_reduction:+.4f}")

# Find best performing model
best_auc_idx = results_df['test_auc'].idxmax()
best_model = results_df.loc[best_auc_idx]
print(f"\n🏆 Best Model (by AUC): {best_model['model']} with AUC = {best_model['test_auc']:.4f}")


COMPREHENSIVE MODEL COMPARISON RESULTS

Performance Metrics Table:
         model  train_accuracy  test_accuracy  train_auc  test_auc  precision  \
0  Ordinary LR          1.0000         0.6286     1.0000    0.5909     0.7619   
1        Ridge          0.6691         0.6857     0.7221    0.5530     0.6857   
2        Lasso          0.6691         0.6857     0.6947    0.5114     0.6857   
3  Elastic Net          0.6691         0.6857     0.5000    0.5000     0.6857   

   recall      f1  
0  0.6667  0.7111  
1  1.0000  0.8136  
2  1.0000  0.8136  
3  1.0000  0.8136  

Relative Improvements over Ordinary Logistic Regression:

Ridge:
  AUC improvement: -0.0379
  Accuracy improvement: +0.0571
  Overfitting reduction: +0.3880

Lasso:
  AUC improvement: -0.0795
  Accuracy improvement: +0.0571
  Overfitting reduction: +0.3880

Elastic Net:
  AUC improvement: -0.0909
  Accuracy improvement: +0.0571
  Overfitting reduction: +0.3880

🏆 Best Model (by AUC): Ordinary LR with AUC = 0.5909
